# Multi-Objective Routing Path Optimization

## Abstract

## Introduction

Search and Optimisation are used in all disciplines of science and engineering, and in wider economy. In economical domains, they have been used in production planning (production mix, machine allocation), corporate finance (capacity planning, working capital management), investments (portfolio optimization, bond/stock portfolio management), and human resources (crew scheduling, office assignment), as well as transportation, agriculture and many others (Zhang, 2024).

In science and engineering, apart from machine learning, they have been used in such fields as office equipment ergonomics, setting construction norms and rules, creating optimal antenna shapes, TCP/IP packet routing, wireless network optimisation, and many others. The problem we have been tasked with in current assessment belongs to the latter – wireless network optimisation. It is one of vital uses of the optimisation methods as wireless networks of many kinds and shapes participate in daily lives of millions of people and practices of many – if not all - businesses.

In the recent decades, it has been instrumental in economic and scientific development of all nations to provide working conditions for wireless data transmission, using GSM (and other) networks. Also, with growing use of Wi-Fi, relay systems had to be optimised. Optimal transmitter locations have had to be established en-masse in order  that the data be accessible in indoor and outdoor locations (F. Aguado Agelet, 2002). A number of methods have been hereby identified; however, researchers elected to use the Nelder-Mead method as it is “easier to implement and appears to be very robust”. Genetic algorithms, however, presented “worse computation-time behaviour”. Choosing the appropriate ways to approach and solve an optimisation problem seems crucial, and understanding the pluses and minuses of each and every one of them can save businesses and research teams a lot of time and effort. Even with separate implementations of the same algorithm, computation complexities, runtimes, and convergence capabilities can vary wildly, as each can have numerous variations, where the balance between exploration and exploitation (of solutions) also varies – as it will be shown in our examples later on.

 Figure 1. A representation of the relation among complexity classes (Wikipedia, 2025)
 ![Alt](images/computation_problems.png)
 
 
Problems of routing, or traversing graphs, loosely related to travelling salesman problem, are examined under the theoretical frameworks of theory of computation, “the branch that deals with what problems can be solved on a model of computation, using an algorithm, how efficiently they can be solved or to what degree” (Wikipedia, 2025). Figure 1 illustrates the relationship between L ⊆ NL ⊆ P ⊆ NP ⊆ PSPACE ⊆ EXPTIME ⊆ NEXPTIME ⊆ EXPSPACE (the first ones being Log-space, Nondeterministic Logarithmic-space, Polynomial Time, Non-Polynomial Time etc.) It is known that “one of the most famous open problems in computer science concerns whether P equals NP” – can every problem whose solution can be quickly verified can also be quickly solved? The current problem of searching for optimal routes ha not found a complete solution that runs in polynomial time, however many optimisation methods have been proposed, and inferences can be made on their computational complexity, efficiency in terms of convergence and suitability for a given problem.

Lance Fortnow gives a neat example of a TSP, whereas a certain saleswoman Mary must analyse the routes of all 48 capitals of contiguous US states (Fortnow, 2013). Whilst she intuitively drew a line connecting them, which is plausibly a “decent” one, there are 48! (48-factorial) solutions to be checked, in regular notation: 12,413,915,592,536,072,670,862,289,047,373, 375,038,521,486,354,677,760,000,000,000 (just over 12 novemdecillion). Concluding, “No wonder Mary’s laptop hadn’t finished in a week”, he goes on to expand that (apart from the proof that N!=NP) we may never find an algorithm that solves the problem all the time:

&nbsp;&nbsp;&nbsp;&nbsp;“We need to rely on other tools, a combination of approximation, heuristics, and computational firepower, simply to do the best we can. NP-completeness gives us a common framework and allows us to create a toolbox of techniques that we can throw at these difficult-to-compute problems.”

Since brute force and/or exhaustive search are an absolute impossibility, other methods for TSP-derived problems can be used. Greedy algorithms are especially useful by allowing us to use ‘ad-hoc’ solutions repeatedly in hope of arriving at a good solution, globally (Thomas H. Cormen, 2001): they must have a “greedy-choice property: a globally optimal solution can be arrived at by making a locally optimal (greedy) choice”; it also needs an “optimal substructure if an optimal solution to the problem contains within it optimal solutions to subproblems”. Of course, a particular optimal route contains optimal sub-routes, therefore Disjktra’s algorithms just works.

Meta-heuristic algorithms are another great option. Many of those are nature-inspired and provide great exploration of the search space in practicable computing time and within limited resources. Some are very ingenuous. Stigmergy is omnipresent in nature and even we humans use the signs (natural or manmade) in the environment to navigate through unknown spaces, and even unknowingly co-operate, not in a dissimilar way from ants in an ant colony (more on ACO algorithm later). For example, writing a note well before a broken footpath, “no throughway” will save anyone who wants to get through in a good amount of time. In addition, some nature-inspired algorithms, like genetics algorithms, produce results that mimic human creativity. David Goldberg expands on the role of chance in directed search, following the musings of the mathematician J Hadamard, “…even though discovery is not a result – cannot be a result – of pure chance, it is almost certainly guided by directed serendipity”, emphasising that the role for chance in human-like discovery is its (blind) ability to “juxtapose different notions” (Goldberg, 2006). He goes on to state that comparisons have been made “with certain human search processes commonly called innovative or creative”. Indeed, it has been pointed out how such pure “chance” in GAs created an antenna for NASA Space Technology 5, where “evolutionary algorithms can be used to search the design space and automatically find novel antenna designs that are more effective than would otherwise be developed” (Gregory. S. Hornby, 2011).

However, GAs’ use in our problem, essentially an optimization of Wireless Sensor Networks, is an overkill as only a tiny fraction of routes will be even valid, potentially leading to a wase of computing resources. Selcuk Okdem (Okdem S, 2009) et al suggest use of Ant Colony Optimization (Okdem S, 2009), for a few reasons, among others that “Although WSNs are used in many applications, they have several restrictions including limited energy supply and limited computation and communication abilities”. Moreover, as “channel bandwidth is limited, protocols should have capability of performing local collaboration to reduce bandwidth requirements”. Here we can proceed to problem statement, where we can look at it closer from a few angles.

### Problem Statement
An organisation has decided to deploy a number of weather sensors communicating wirelessly, between one another, and two base stations, in New Forest area of England. The resources are limited and there are bandwidth constraints, and related latencies in communication. The table below explores the technical abilities of the said sensors, where distances between them dictate the network performance. There are 150 units randomly within an square area, where base stations are at (-5000, 5000) and (5000, -5000), and the sensores are anywhere in between. Geo data has been provided by the organisation in the form of a csv file. Eucladean distances have to be calculated from the formula for d, as below.
$$d = \sqrt{(x2 - x1)^2 + (y2 - y1)^2}$$
Having done that, at least two different algorithms must establish the best routes for each sensor to a base station (relaying via other sensors), indexed from 0 to 149, taking into consideration the communication efficiencies.
The bandwidth between the sensors is based on the distance between them. The bandwidth-distance data is shown in the table below.

| Distance d | Transmission rate (Mbps) | 
| ------ | ------ |
| d >= 3000m | 0 |
| 3000m > d >= 2500m | 1 |
| 2500m > d >= 2000m | 2 |
| 2000m > d >= 1500m | 3 |
| 1500m > d >= 1000m | 4 |
| 1000m > d >= 500m | 5 |
| 500m > d | 7 |

## Methodology

The problem as described, does not lend itself to a straightforward one-size-fits-all solution. It is a problem of two seemingly opposite objectives. To bring up an example from real life, buying a motorcycle may be an exercise in a multi-objective optimization: price, vehicle weight-to-bhp ratio, comfort (seat height, handlebar-rider’s arm angle), acceleration 0-60, fuel use, and top speed may all be factors. If the motorcycle were to be used for commuting long distances through city traffic, comfort, fuel use, vehicle weight and (perhaps) acceleration would be priorities, whilst top speed would be nearly irrelevant. Used as a second vehicle for weekend fun on country roads, or track-days, acceleration, weight-to-power ratio, and top speed would come first, comfort would be less viable, and fuel use – irrelevant (assuming enough financial means). 

The only way to solve so many objectives is sometimes to put weights on all variables with a weighted sum method (see Figure 2) (Zhang, 2024)

 ![Alt](images/Multi_fun.png)
Figure 2. Multi-objective function conversion to a single objective function using weights.
 

 Some functions need to be maximised, and some – minimised; in everyday parlance, some denote a ‘good thing’, and some a ‘bad thing’. In our problem example, latency is our ‘bad thing’, which needs minimising all along, whereas edge-to-edge transfers have to be maximised on each potential route, thus avoiding the edges that constitute unnecessary bottlenecks on the way.
 
Utilising reciprocals is a good way to achieve this end in practice, including here. We have decided together, upon consultation with the lecturer, to use this method, and upon a trial-and-error process, we settled on certain weights, 1 being the “alpha” and 150 – the “beta”.

### Objective Function

```f(x,y)= α⋅x + β⋅1/y,```

Where:

```f(x,y) - objective function```

```α - weight given to variable x```

```β - weight given to the reciprocal of variable y```

```x - minimum edge transfer on the route (“bottleneck”)```

```y - sum of latencies on the route.```

For obvious reasons, the transfer on any “bottleneck” should be as high as possible, therefore the variable x needs to be maximised. Conversely, the sum of on-route latencies must be minimised, hence the use of the inverse variable. The entire objective f(x,y)  function shall be maximised for every route from each node to either base station, whichever is better. The algorithms make ad-hoc stochastic choices on which base station should be chosen, and optimal one is always found, along with a route that (potentially) is the optimal one. The use of potentially is intentional. Only gradient-based methods based on well-defined functions yield a 100% guarantee of nice and accurate result of optimisation, all other methods being on a spectrum, whose other far end is lack of convergence. These often get put into one ‘bag’ named ‘metaheuristic’ (discussed in the work as cited):

The ongoing improvement of heuristic algorithms often gets referred to as “metaheuristics” despite the lack of a unified scientific definition. (Selvarajan, 2024)

Perusing the paper, and unit lectures allowed us to construct a very simple ranking of optimisation methods that we considered for solving the problem. Some methods lack merit completely, as gradient methods, as there is no neat well-define mathematical objective function to optimise. Similarly, linear programming would be ill suited. We are left - more or less - with metaheuristic methods.

<b>It must be noted that the work has been done by a number of students. Wherever an author means "absolute cost" of a path or route, it shall mean "objective function value" on that path/route, as per the definition above**. Let us remember that this 'loose' use of the word 'cost' will allow us to think of it in terms of benefit, rather than traditional meaning of 'cost', and when the "absolute cost" is maximised, it shall mean fulfilling the objectives of our work, and indeed a positive outcome.</B>

**(except PSO, which has a different objective function, added under its specific section).

### Selection of Methods
Let us have a glance at what we can choose from (Figure 3)

![Alt](./images/rank_of_methods.JPG)

	1. Gradient-based Methods: High accuracy and reliable convergence.
	2. Quasi-Newton Methods: Similar to gradient-based methods but may require adjustments.
	3. Convex Optimization: Generally reliable, but not as precise as gradient-based methods.
	4. Linear Programming: Useful in many situations but may face convergence issues in complex cases.
	5. Dijkstra's Algorithm: Typically, reliable for shortest path problems.
	6. Ant Colony Optimization (ACO): Often used for finding optimal paths, good at exploring diverse solutions.
	7. Simulated Annealing: Good for escaping local optima, but convergence can be slow and uncertain.
	8. Differential Grouping Algorithm (DGA): Useful in decomposition-based optimization.
	9. Genetic Algorithms: Offers diverse solutions but can lack convergence and consistency.

For our purposes, we shall be using Dijkstra’s Algorithm, ACO, Simulated Annealing and a variant of Genetic Algorithms (DGA). One student proposed his own solution with a Particle Swarm Optimization (PSO) method, which is an interesting take on this algorithm.

Ant Colony Optimization (ACO): The paper (Selvarajan, 2024) highlights ACO's strength in exploring diverse solutions and its biologically inspired approach; it mimics the foraging behaviour of ants. It is noted for good exploration capabilities, but may require fine-tuning for specific problems. This will be performed and discussed in ACO documentation in more detail, together with elementary statistical analysis of the ACO parametrisation process.

Simulated Annealing: This method is praised for its ability to escape local optima by simulating the annealing process of metals. However, it is mentioned that convergence can be slow and is not guaranteed in all cases.

Dijkstra's Algorithm: The paper (Selvarajan, 2024) discusses Dijkstra's Algorithm as highly reliable for shortest path problems in graphs. It is noted for its accuracy and guaranteed convergence but is limited to specific types of optimization problems. It seems to be particularly well suited to our problem.

Genetic Algorithms (GAs): Genetic Algorithms are recognized for their robustness and ability to handle complex search spaces. They are good at exploration but may lack consistency and convergence in some scenarios. We must note their apparent stochastic ‘creativity’, or rather, a wide space of solutions tested. Of course, in our scenario, many routes created will make no ‘sense’ as there will be no such connections / edges in our graph.

Particle Swarm Optimization (PSO): We looked at the available papers, which adapt PSO, which is more suited to optimising non-differentiable and continuous-value functions rather than routing problems, to the extent that we found a solution. Implementation of PSO is attempted by a student, together with relevant documentation.




### Extra Requirements

To be able to execute the algorithms and visualiza the data, there was one extra dependency we required to import to our conda environment. This was the `Folium` library which is map visualization API that we used to plot our solutions. Depending on your environment, it could be installed using either of the following shell commands. (If you are using pip package manager, uncomment the 'pip command in the second code cell below and comment the conda command).
```sh
! conda install -y conda-forge::folium
```
or
```sh
! pip install-y folium
```

In [10]:
#!conda install -y conda-forge::folium

In [11]:
#!pip install folium

### Code Modularization

For simplicity, the code was modularized based on functionality. The modules and their respective functionalities are as follows:<br>
- `data_plot`: Has all functions that fetch data from CSV file, preprocesses data into forms that are required by the various algorithms and visualizes the results of an algorithm on a map
- `metrics`: Consolidates the algorithms evaluation functions using the 2 metrics, 'execution time' and 'solution quality'
- `discrete_genetic_algorithm`: contains the discrete genetic algorithm implementation
- `ant_colony_optimization`: contains the ant colony optimization implementation
- `dijkstras_algorithm`: contains the dijkstra's algorithm implementation
- `simulated_annealing`: contains the simulated annealing algorithm implementation
All these algorithms and functions are accessible in this report and will be called using a single function with the type of algorithm input and extra optional optimization **kwargs accepted as parameters. The execution of the modules is done below.

#### Imports

In [15]:
import ast
import json
import scipy.stats as stat
%run data_plot.ipynb
%run metrics.ipynb
%run discrete_genetic_algorithm.ipynb
%run ant_colony_optimization.ipynb
%run dijkstras_algorithm.ipynb
%run simulated_annealing.ipynb

This is alongside some import which are required for executing of some functions in this main module. For example, the json import is used in reading and writing execution results in the solutions json file.

### Results Visualization on a Map

To visualize the solutions produced by the various algorithms, we implemented a real and interactive geographic map with the sensors and the base station. This was done using Folium, a Leaflet JavaScript library which we interacted with through an API. We followed through the API documentation (Story 2013) to achieve the following:
1. Show the best route obtained from a sensor to a base station by a particular algorithm
2. Show all other possible routes found (not the best but have end-to-end connectivity)

The locations of the sensors were provided in form of X, Y grid coordinates in meters for purposed of calculating their heuristic distances. However, since the API needs values of geographic coordinates, coordinates of the sensors had to be computed and represented on the leaflet. The computation of geographic coordinates from X, Y grid coordinates was inspired by the Haversine formula (Scripts 2024).

`latitude = central latitude + y/110574`<br>
`longitude = central longitude + x/(11320 * cos(central latitude))`

where x and y are the x, y values of the sensors provided in the csv file while central latitude and central longitude are the (0, 0) coordinates on the grid from which all other sensor distances are calculated from. From the map provided in the graphic in the assignment instructions we estimated the geographic locations of the base stations in Lyndhurst and Beaulieu as shown in figure 1 by referring to google maps.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/search%20area.png?raw=true)

The values ‘110574’ and ‘11320’ in the equations above are the distances between the latitudes and longitudes in meters respectively (Rosenberg 2024). With the geographic coordinates derived from the grid coordinates, we were able to feed the API with the sensor and base station information which was useful in plotting paths.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/blank%20map.png?raw=true)

The visualization code functions are found in the [data and plotting module](./data_plot.ipynb) which is has all data fetching and manipulation functions as well as the map visualization functions.

#### Map Visualization Functionalities

<b>Data Fetching and constant variable initialization:</b> The X, Y grid data is fetched from the CSV file and all constants related to the data such as the ‘distance bandwidth data’ defined in the instructions are initialized. It is at this point that a blank map in the search area of interest is initialized.<br>
<b>Grid to Geographic coordinates conversion:</b> Using the formulas defined above, the X, Y data is converted to geographic coordinates using the function `get_geographic_coordinates()`<br>
<b>Initialization of a map with sensors:</b> Using the computed geographic coordinates, the sensors and base stations are represented on the blank map. This initialization happens every time an algorithm is executed for visualization to be done on a fresh map. This is done using the `init_blank_map()` function.<br>
<b>Plotting routes:</b> Joins the sensors that are part of a transmission path using lines from the origin node to the destination node. This is done for the best path found which is represented using a yellow line and all the other possible paths that are represented using a variety of colors. The ‘best path’ and ‘possible paths’ have been categorically layered in a way that you can view each type independently without struggle. For further visual intuition, the best path’s sensors have been marked with pins which show their sensor IDs. This can be done using a layer controller found on the top left corner. This functionality has been implemented in the `plot_route()` function. With takes in the coordinates as arguments with a specification of whether they are the best routes or they are possible routes.


![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/map%20visualization.png?raw=true)

<b>Parsing data for visualization:</b> In the main module (this report), the main function `find_optimal_route()` returns information on all <i>valid routes</i> discovered and the <i>best route</i>. This infromation is used for visualization on the map. Before this is done, the data need to be converted to a format that the `plot_route()` function described above takes. An example of the format is as follows:
```
coordinates = {'best_route': {'sensor_id': [19, 28, 145],
              'coordinates': [(50.83182308300549, -1.50844786872212),
               (50.821231533454956, -1.4924287442749806),
               (50.821231533454956, -1.4924287442749806),
               (50.80703108850407, -1.4877640416052273),
               (50.80703108850407, -1.4877640416052273),
               (50.817205047239135, -1.4493866785949845)]}, 
               'possible_routes': [[(50.79834911986768, -1.4912597403722938), (50.82619020366676, -1.5019504696113806),
                                    (50.82619020366676, -1.5019504696113806), (50.817205047239135, -1.4493866785949845)],
                                   [(50.79834911986768, -1.4912597403722938), (50.80721558033759, -1.5461448507014692),
                                    (50.80721558033759, -1.5461448507014692), (50.817205047239135, -1.4493866785949845)],
                                  ]
              }
```
To achieve this, the `parse_map_data(results)` is called with the results of the execution passed in and the data is converted in that format. The function is shown below.
<br>

In [29]:
def parse_map_data(results):
    """
    parse_map_data(results)
    converts routes data to geographic routes that can be represented on a map.
    this is done by checking the sequence of x y coordinates against their corresponding
    geographic coordinates defined in the plotting module.
    results: the algorithm search results that contains the best solution to the destination
    and all other possible routes.
    returns: the coordinates used to create the map representation
    """
    # init dictionary with the structure expected for map visualization
    coordinates = {'best_route': {'sensor_id': [],
                              'coordinates': []}, 
               'possible_routes': []
              }
    # convert the string containing the list of the best path to type list
    possible_paths = [ast.literal_eval(paths) for paths in results['possible_paths'].keys()]
    # get the geographic coordinates for all nodes in possible paths
    for path in possible_paths:
        route_coords = []
        possible_path_dest_idx = len(path) - 1
        for idx, sensor in enumerate(path):
            if idx != 0 and idx != possible_path_dest_idx:
                # duplicate coordinates for all nodes but for the first and last
                route_coords.append(geographic_coords[str(sensor)])
                route_coords.append(geographic_coords[str(sensor)])
            else:
                route_coords.append(geographic_coords[str(sensor)])
        coordinates['possible_routes'].append(route_coords)
    # add best path last so that an visual it appears on top
    best_path = ast.literal_eval(list(results['best_path'].keys())[0])
    best_path_dest_idx = len(best_path) - 1
    for idx, sensor in enumerate(best_path):
        # include sensor IDs for the best path except for the destination (won't need pin marker)
        if idx != best_path_dest_idx:
            coordinates['best_route']['sensor_id'].append(sensor)
        if idx != 0 and idx != best_path_dest_idx:
            # duplicate coordinates for all nodes but for the first and last
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
        else:
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])

    return coordinates

### Saving best results in a solutions file

After execution of each algorithm using the `find_optimal_route()` function, we had a best solutions variable as one of the returned values of the function. This information that takes the format shown below is stored in a json file named `solutions.json`.
```
{"discrete genetic algorithm": {"Node-3": {"source node": "Node-3", "routing path": "(Node-123, 3 Mbps),(Node-152, 5 Mbps),", "end-to-end transmission rate": "3 Mbps", "end-to-end latency": "60 ms", "absolute path cost": "5.5"}, "Node-5": {"source node":...
```

This file contains information on the best solution provided by each algorithm for each node. The parent dictionary contains the four algorithms implemented and the sub-dictionaries under each algorithm contain the 150 nodes and their best solutions to either base stations. To implement this, the information returned by the `find_optimal_value()` function is taken through a write function called `save_solution(solution, method)` that takes in the solution and the method/algorithm used as parameters. This function further utilizes a `load_solutions()` function that gets all existing solutions from the json file, check if the solution for the algorithm exists and if it exists, it overwrites with the new value, otherwise it creates the sub-dictionary and records the new best solution to that node. The two functions are shown below.

In [33]:
def load_solutions():
    """
    load_solutions()
    it loads solutions that already exist since new solutions are appended to the json
    object before being added written on the file.
    returns: the json object read from the file
    """
    # fetch existing best solutions from the solutions file
    with open('solutions.json', 'r') as file:
        try:
            solutions = json.load(file)
        except Exception as e:
            # if the solutions file is empty, initialize and empty dict to be used
            solutions = {}

    return solutions

In [34]:
def save_solution(solution, method):
    """
    save_solution(solution, method)
    saves the best solution found by each algorithm in a solutions json file.
    the solutions file has a format like...
    {"source node": "Node-5", 
    "routing path": "(Node-2, 1 Mbps), (Node-1, 4Mbps), (BS-2, 2 Mbps)",
    "end-to-end transmission rate": "1 Mbps"}
    solution: the best solution to a base station from a sensor
    method: the method that was used to get that best solution.
    returns: nothing
    """
    # fetch existing solutions
    existing_solutions = load_solutions()
    '''
    if solution for a sensor using the method specified exists, 
    update the dict with a new solution, otherwise create the solution
    '''
    if method in existing_solutions.keys():
        existing_solutions[method][solution[0]['source node']] = solution[0]
    else:
        existing_solutions[method] = {}
        existing_solutions[method][solution[0]['source node']] = solution[0]
    # write the updated solutions to the solutions file
    json.dump(existing_solutions, open( f"solutions.json", 'w' ) )

    return

### Algorithms

To solve the routing problem, the following algorithms were implemented:
1. Discrete Genetic Algorithm
2. Ant Colony Optimization
3. Dijkstra's Algorithm
4. Simulated Annealing Algorithm

The above algorithms have been packaged into a single optimization function `find_optimal_route(origin, method, **kwargs)` that is executable in this report. The function shown below.

In [37]:
def find_optimal_route(origin, method, **kwargs):
    """
    find_optimal_route(origin, method, **kwargs)
    the main function of this assessment. It searches for best route 
    from a node to either of the 2 base stations. The search is done using
    the algorithm specified as one of the arguments. Supported algorithms 
    include: 'discrete genetic algorithm', 'ant colony optimization', 
    'dijkstras algorithm' and 'simulated annealing'. Each of the algorithms,
    return a best solutions dictionary that is stored in the solutions file,
    and the results dictionary which is used for visualization on a map
    origin: the node whose best path to a base station you are seeking
    method: the algorithm you want to use to perform the search
    **kwargs: any additional arguments you want to pass in for
    optimization of the various algorithms
    Expected **kwargs and their fallback values:
    discrete genetic algorithm: pop_length=100000, generations=10, mutation_rate=0.1, crossover_type='single-point', selection_limit=20
    ant colony optimization: Q=1, alpha = 0.6, evaporate=0.45, ants_count=20, max_iterations=10
    dijkstras algorithm: None
    simulated annealing: init_temp=100, final_temp=0.1, cooling_rate=0.95, improvement_checker_count=50
    More details on the **kwargs provided in the algorithms' respective modules
    returns: nothing
    """
    # run the algorithm passed in as a parameter and fetch the visualization results together with the solutions file data
    if method == "discrete genetic algorithm":
        results, solutions = discrete_genetic_algorithm(
            origin,
            pop_length=kwargs.get("pop_length", 100000),
            generations=kwargs.get("generations", 10),
            mutation_rate=kwargs.get("mutation_rate", 0.1),
            crossover_type=kwargs.get("crossover_type", "single-point"),
            selection_limit=kwargs.get("selection_limit", 20),
        )
    elif method == "dijkstras algorithm":
        results, solutions = ant_colony_optimization(
            origin,
            Q=kwargs.get("Q", 1),
            alpha=kwargs.get("alpha", 0.6),
            evaporate=kwargs.get("evaporate", 0.45),
            ants_count=kwargs.get("ants_count", 20),
            max_iterations=kwargs.get("max_iterations", 10),
        )
    elif method == "ant colony optimization":
        results, solutions = ant_colony_optimization(origin)
    elif method == "simulated annealing":
        results, solutions = simulated_annealing(
            origin,
            init_temp=kwargs.get("init_temp", 100),
            final_temp=kwargs.get("final_temp", 0.1),
            cooling_rate=kwargs.get("cooling_rate", 0.95),
            improvement_checker_count=kwargs.get("improvement_checker_count", 50),
        )
    else:
        return "unknown algorithm"
    # save the best path in the solutions file
    save_solution(solutions, method)
    # parse the plotting results and call the map visualization function
    coordinates = parse_map_data(results)
    plot_route(coordinates)

    return

In [38]:
find_optimal_route(43, 'discrete genetic algorithm')
#find_optimal_route(43, 'ant colony optimization')
#find_optimal_route(43, 'dijkstras algorithm')
#find_optimal_route(43, 'simulated annealing')

**********************************Generation 1**********************************
********************************************Performance**********************************
Chromosome: [ 43  77 151]-----actual route: 43->77->151->----Score (end-to-end path cost): 3.5
Chromosome: [ 43 145 151]-----actual route: 43->145->151->----Score (end-to-end path cost): 4.5
Chromosome: [ 43  15 151]-----actual route: 43->15->151->----Score (end-to-end path cost): 3.5
Chromosome: [ 43 103  72 151]-----actual route: 43->103->72->151->----Score (end-to-end path cost): 2.667
Chromosome: [ 43  61 151]-----actual route: 43->61->151->----Score (end-to-end path cost): 3.5
Chromosome: [ 43  97 151]-----actual route: 43->97->151->----Score (end-to-end path cost): 3.5
Chromosome: [ 43  45  35 129  61 151]-----actual route: 43->45->35->129->61->151->----Score (end-to-end path cost): 2.0
Chromosome: [ 43  15  53 151]-----actual route: 43->15->53->151->----Score (end-to-end path cost): 2.667
Chromosome: [ 43  72 1

#### Discrete Genetic Algorithm (DGA)

This is a stochastic algorithm that was used in seeking the best solutions (paths) from a particular node to either base station. It uses the classical genetic algorithm approach with the following steps:
1. Population initialization
2. The selection process that encompasses fitness function evaluation
3. Crossover
4. Mutation
These steps are done iteratively until our termination criteria which is ‘maximum number of generations’ is reached. The full code implementation of this algorithm is found in the [discrete genetic algorithm module](./discrete_genetic_algorithm.ipynb). The specifications of the steps above are explained in the process flow below.

##### Process Flow

<b>Population Initialization:</b> The algorithm starts with random initialization of the population which in this case is the various routes from the specified origin to a specified destination (base station). This is done in the function `dga_init_population()`where the number of routes generated is also specified as an argument. The function generated random values excluding the origin and destination regardless of whether they have end connectivity and at the end prepending and appending the origin and destination respectively. The validity of the paths will be checked in subsequent stages. A list of randomly generated routes is returned form the function.<br>
<b>Fitness Function Evaluation:</b> The next stage is calculating the end-to-end fitness function which incorporates the end-to-end bandwidth and total latency using the cost function defined for this project. This is done in the `bandwidth_latency_calc()` function and while doing all this, it updates the global variables ‘solutions’ and ‘dga_data’ which carry map visualization data and best solutions (for the solutions file) data. The function returns the population (routes) with their corresponding end-to-end fitness function values.<br>
<b>Filtering out valid paths:</b> The next stage is filtering out valid paths from the ones that are created. Since the algorithm has no specified formulae in the way the path is created, many invalid paths have a high chance of being created. Therefore, there is need to exclude them so that the subsequent stages run on quality routes. This process is run in the `get_valid_paths()` function. The function eliminates any chromosome with an end-to-end fitness value of 0 as this means there is no end-to-end connectivity. It is at this stage that we determine if there is no valid path completely to a particular station that has been generated. If that is the case, the algorithm reruns from scratch and seeks a path to the alternative base station using the processes described above.<br>
<b>Fitness Selection Process:</b> Using the valid paths determined by the previous stage, a selection process is carried out using the tournament selection method. Here the selection is based on the solutions that have the highest fitness function values since our main aim is to maximize the values. This is carried out in the `tournament_fitness_selection()` function which has a set number of maximum possible winners in the function parameters. The result of this function is quality solutions which will be improved in the subsequent stages. If the quality solution left is just 1, the solution is returned without proceeding further.<br>
<b>Crossover:</b> Just before this, the process of preparing the chromosomes for crossover is done. This entails pairing best solutions in 2s since the crossover process requires pairs of chromosomes. This is alongside the type of crossover (single-point/ two-point/ uniform) as arguments to the function `crossover()` where the process happens. The result of the process is 2 children with mixed genes (nodes/sensors in the routes) that could potentially be better solutions than their parents.<br>
<b>Mutation:</b> This process seeks to introduce new information to the gene. In the shortest route problem that we seek to solve, mutation is done in the form of swapping genes (sensors) for 2 reasons:
1. There is a limited number of valid sensors that can be introduced to the solution
2. We aim at having sensors with valid end-to-end connectivity values.

The number of swaps made is determined by the mutation rate defined in the function `mutate()` where this entire process runs. The nodes that are to be swapped exclude the origin and destination/base station nodes which are the fixed.

The above processes are consolidated into the function `dga_run()` which takes in all the requires parameters. The parameters are:
1. `origin`: the origin from which you want to traverse the space.
2. `destination`: the destination node which you want to get to.
3. `population length`: the length of the population for the first generation.
4. `generations (optional)`: the maximum number of generations that you would want to you do work with (termination criteria). Default is set to 10.
5. `mutation rate (optional)`: The percentage of the population that will be mutated. Default is set to (0.1).
6. `crossover type (optional)`: The type of crossover that will be implemented. The default is set to 'single-point'.
7. `selection limit (optional)`: The number of winners that should be selected at the minimum during the fitness selection stage. Default is set to 20.

The algorithm is tested for each base station. The population generated point to a certain base station and if a valid solution to that base station is found, the alternate base station is tried. This is done using a try-except statement in the wrapping function `discrete_genetic_algorithm()` which is the main function of this algorithm and is called in the main function file. The result of this function is the plotting data that has the best and possible paths alongside the solutions variable that has the data of the best path that will be recorded in the solutions file. The above processes are illustrated in the flow chart below.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/dga%20flowchart.png?raw=true)

To execute the discrete genetic algorithm, run `find_optimal_route(sensor, 'discrete genetic algorithm', **kwargs)`. For example:

```
origin_sensor = 43

dga_kwargs = {"pop_length": 150000,
              "generations": 20,
              "crossover_type": "single-point"
            }
find_optimal_route(origin_sensor, 'discrete genetic algorithm', dga_kwargs)
```

#### ACO - Ant Colony Optimization

Introduction

This algorithm was used in searching for the optimal path from any node to either of the base stations. The algorithm employs ‘ants’ by the way of iterative process, in a stigmergic fashion working ‘together’, although not concurrently or in parallel (we looked into these but deemed them redundant), marking paths with pheromones for more ‘ants’ to follow in following iterations. The Elitist ACO approach adds an extra pheromone level for the ant discovering the best path. Higher graph edge’s pheromone level may increase the probability of an ant transition to it, but the process is still stochastic by utilization of a roulette wheel method, increasing ants’ chances at exploration and preventing the model from being too deterministic. Evaporation has been used in both algorithms. Global pheromone update has been used. The ACO code can be followed in the [ant colony optimization module](./ant_colony_optimization.ipynb).

#### Process Flow, Methodology

Routing Table Creation: The algorithm starts with the creation of an adjacency matrix to represent the geographical data (from co-ordinates). This is done by calculating the Euler distances and the bandwidth connectivity between the nodes (acting as graph weights) as per the Mbps table supplied. This matrix will be essential in creating random routes that have end-to-end connectivity at the fastest rate possible. One algorithm (RH) used out_nodes and out_pheros structures, to ease the coding process, interpretability, and readability. They are essentially a graph adjacency list, and its auxiliary matrix of pheromones, respectively. RH algorithm used the Elitist approach, wherein the elite ant left a 3 x higher pheromone than regular ants. SM code used classic ACO solution, with equal weight given to all ants’ pheromones. In SM implementation, ants were deployed not only from the origin node (as in RH code), but also some adjacent nodes, to improve exploration; he called them ‘helper ants’ – it is a novel idea. Both implementations used the version of calculating probability of single transition to another edge in the graph with the value of tau to the power of alpha, as well as eta to the power of beta divided by their summation. Although it arguably adds some pre-determination into the method, it possibly allows us to reduce the number of iterations and save on computing power. Evaporation rate was 0.45 in both, striking a balance between the persistence of emerging route solutions (pheromone staying power), and stochastic exploration of new / poorly explored unexplored paths. The cumulative probability in both programs was used to construct a roulette wheel, where randomness introduced an extra level of exploring the search space. When routing to one destination or the other, a pseudo-random function was utilized. Both algorithms use loop removal, implemented in slightly different ways.
Two operations are weighty for the algorithm’s workings, i.e. positive and negative feedback. Positive feedback takes information from the ants in order to apply stigmergy to the system.  After all ants within an iteration have constructed their paths from the source to destination and graph cycles are removed, the pheromone intensity on an edge (i, j) is adjusted:

$$
\tau_{ij}(t + 1) = \tau_{ij} + \sum_{k = 1}^{n_k} \Delta \tau_{ij}^k(t)
$$
where:
$$
\Delta \tau_{ij}^k(t) ) = \begin{cases}
    \frac{Q}{f(x^k(t))} & \text{if edge (i, j) occurs in path x}^k(t) \\ 
    0 & \text{otherwise.}
\end{cases}
$$
$x^k(t)$  is the solution of ant k<br>
$f(x^k(t))$ is the quality of the solution (cost of the path)<br>
Q > 0 and is a constant<br>
$n_k$ is the number of ants<br>
<br>
We had to correct one of the programs in order to reflect the fact that our objective function is not strictly a ‘cost’ function, but a ‘benefit’ function to be maximized. Hence, the function value was hereby raised to a power of positive, rather than negative, exponent (here: 1 rather than -1) It enhanced the algorithm, although it still seemed to work and converge satisfactorily well even before that correction. 
Negative feedback was a simple run through the pheromone matrix by multiplying a scalar (1- ϱ) by it. In both apps, we used 0.45 and it was found effective.
The only stopping criteria we used is the number of iterations. Another could be used, too: all (or most) ants follow the same path. It could lead to waste of computational power in non-critical solutions, though. We shall expand on it in the next part.

#### Parameter Choice, Experiments and Performance Analysis

Having coded, troubleshooted and tested the code, it was time for analysis of the solution files and fine-tuning the parameters. RH code initially set the parameters to the same levels as SM (ants_n=20, iterations=10), but performed many tests and performance and quality analysis to establish a good balance for the apploication domain (weather monitoring system). A chat with OpenAI’s ChatGPT was used, provided with 9 detailed solution files and timings (ChatGPT, 2025). Increases from 10 to 15 and then 20 iterations offered merely modest improvements in the quality of solutions measured by mean objective function for all routes in 10, 15, 20 iteration version (+4.39% and +4.30%, respectively, and only the latter is marginally statistically significant); however, the timings went up from 3 to 5 then 7 minutes per execution. The tests for each of the three iteration parameters were repeated 3 times, and no difference in run time or solution quality was noted. Similarly, increasing the number of ants from 20 to 30 did not affect either the run times, or the quality of the solution.
The recommendation for our optimization problem would be to persist with the lower values of iterations as sufficiently accurate and more computationally effective. The increase from 15 to 20 iterations, while “marginally significant, it may justify the cost in high-stakes scenarios”, like high precision or critical systems.

#### Tweaks and flowchart

We added the following tweak to the transition probability formulae, that is also commonly used in travelling salesman problem (which shares similarities with and differences from ours).

$$\large
P^k_{i,j} = \begin{cases}
    \frac{\tau^\alpha_{i,j}(t)\eta^\beta_{ij}(t)}{\Sigma \tau^\alpha_{iu}(t)\eta ^ \beta_{iu}(t)} & \text{if j }\in{ N^k_i(t) }\\ 
    0 & \text{otherwise.}
\end{cases}
$$
for $\mathbf{k} = \langle 1, 2, \dots, n_{k}\rangle$<br>
where: $\tau_{ij}(t)$ is the pheromone intensity<br>
	   $\eta_{ij}(t)$ is a priori effectiveness of the move from i to j<br>
	   $\alpha\gt0, \beta\gt0$ are predefined constraints<br>
	   $\eta_{ij}(t) = \frac{1}{d_{ij}(t)}$ improves the attractiveness of the edge (i, j)<br>
	   $d_{ij}(t)$: cost between edge (i, j)<br>

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/aco%20flowchart.png?raw=true)

#### Dijkstra's Algorithm

This report discusses the implementation of Dijkstra's algorithm according to what was specified in the provided notebook. The method has been tailored for network optimization, emphasizing variables such as latency, bandwidth, and overall cost-effectiveness. The full code implementation can be followed in [dijkstra's algorithm module](./dijkstras_algorithm.ipynb).

The objective of this implementation is to come up with the most appropriate routes between nodes in a network by:
- Optimizing bandwidth, or data transmission rates.
- Cutting down on latency, or delays.
- Balancing overall cost considerations for routing.

#### Key Components

Variables:<br>
`dijkstra_data`: Contains information about the best and alternate paths.<br>
`known_costs`: Stores the current best-known costs to each node.<br>
`routes`: Records the path taken to reach each node.<br>
`visited_nodes` and `unvisited_nodes`: Track the traversal status of nodes.<br><br>
Initialization Function:<br>
`init_known_costs(init_node)`:
- Sets the cost of the initial node to zero.
- All other nodes are initialized with unlimited cost values.
- Sorts the nodes into groups that have been visited and those that have not.

#### Path Computation Functions

##### Bandwidth and Distance Calculation
`get_path_bandwidth_costs(current_node)`:
- The bandwidth between the present node and its neighbors is calculated.
- Uses helper functions:<br>
`calculate_distance()`: calculates the geometric separation between nodes.<br>
`convert_distance_bandwidth()`: converts distances into bandwidth values that are equivalent.<br>

##### Node Selection Logic
`select_max_path(node, no_visited_nodes)`:
- The node that has the highest absolute cost is chosen as the next one to investigate.
- The lists of visited and unvisited nodes are updated appropriately.

#### Route Optimization and Results

##### Optimal Path Identification
`get_optimal_route(destination, dijkstra_data, solutions)`:
- Finds the most efficient route between the starting and finishing points.
- Updates dijkstra_data with the results for analysis and display.
- Adds important metrics to solutions, such as:
1. source and destination nodes.
2. Path sequence.
3. Values for bandwidth and delay.
4. The total expense of the path.


##### Algorithm Workflow

`dijkstra_algorithm(init_node)`:
- Executes the fundamental reasoning behind Dijkstra's algorithm.
- Updates path data and calculates optimal costs iteratively.

#### Supporting Data
##### Node and Network Information
Node Coordinates:
- x_y_data: List of coordinates for network nodes.
- x_y_base_station_1 and x_y_base_station_2: Positions of key base stations.

#### Cost and Latency Metrics
- node_to_node_latency: Captures time delays between nodes.
- absolute_cost_function: Calculates the combined cost of bandwidth and latency.

The flowchart showing the processes described above is shown below:<br>
![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/Dijkstras%20Flowchart.png?raw=true)

#### Simulated Annealing

This algorithm was used in seeking for the best path from a node to any of the base stations. This was done by iteratively checking the absolute cost of the random paths generated up to a point when the following 2 conditions have been met:
1. The temperature cools to the lowest set temperature for the algorithm
2. There has been no considerable improvement in the absolute cost over the course of several iterations.
For purposes of exploration of the search space, the probability of the algorithm picking a worse off solution is high at the initial phases (when the temperature is high) but in later iterations, better solutions are highly considered.

The complete code for this algorithm can be followed in the [simulated annealing module](./simulated_annealing.ipynb).

##### Process Flow

<b>Routing Table Creation:</b> The algorithm starts with the creation of a connectivity graph. This is done by calculating the distances and the bandwidth connectivity between the nodes. This graph will be essential in creating random routes that have end-to-end connectivity at a fast rate. This process is implemented by the `calculate_route_costs ()` function which is fed in the x_y_data that contains the x, y positions of the sensors. A 2D array with the calculated bandwidth information is returned.<br>
<b>Initial Route Generation:</b> A random route in created and to ensure validity, the routing table created in the previous step is utilized. The path generated could be of varying length and could be leading to any of the base stations. This is done concurrently with the route absolute cost calculation. This is done in the `generate_route ()` function which takes in the origin node and the routing table data. The absolute cost calculation is done in the `bandwidth_latency_calculation ()` function with takes in the generated route. The route with its corresponding cost is returned from the whole process.<br>
<b>Variable Initialization:</b> This is a stage where variables like the ‘starting temperature’, ‘final temperature’, ‘cooling rate’, and ‘improvement checker count’ are initialized. These are the variables that control the functioning of the main function called `simulated_annealing ()`. Their functions are defined below:
1. starting temperature: the start temperature defined for the simulated annealing algorithm
2. final temperature’: the final temperature defined for the simulated annealing algorithm
3. cooling rate: the temperature cooling rate for the algorithm for the algorithm 
4. improvement checker count: the number of iterations to go through before checking if there is any sensible improvement in the path quality
   
<b>Recursive Exploration:</b> The algorithm then gets into the core functionality of generating other possible solutions and checking if they are better than the current best solution. Initially, the first solution is set as the best solution and thereafter, any solution that is better than that is used as the best solution for forthcoming iterations. Meanwhile, the cooling of the temperature happens at the set rate. During the initial iterations, the probability of a worse of solution being used as an acceptable solution is high since the high temperature is used in calculating the probability of a path being picked. As the temperature goes down, the influence of temperature in determining whether a solution will be accepted is low. The formulae used in determining whether a solution is picked is as follows:<br>
`delta = current path cost – best path cost`<br>
`probability = exponential (-delta / current temperature)`<br>
Just like in a roulette wheel, a random number is generated and the side in which is falls in determines the solution’s acceptance. In this case, if the number is less than the calculated probability, the worse off solution is accepted.
This loop goes on until the lowest temperature is reached. However, I a situation where several iterations have been done but there is no significant improvement, the loop is broken and the solution accepted for purposes of saving time and computation resources.

The main function returns the generated solutions that is categorized as ‘best routes’ and ‘possible routes’ as is a standard with all the other algorithm implementations for visualization purposes. The solutions variable containing the best path in the format required by the solutions file is also returned. The process flow is represented in the flow chart below.


![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/simanneal%20flowchart.png?raw=true)

To execute the discrete genetic algorithm, run `find_optimal_route(sensor, 'simulated annealing', **kwargs)`. For example:

```
origin_sensor = 43

sim_anneal_kwargs = {"init_temp": 70,
                     "final_temp", 5,
                     "cooling_rate", 0.82
                    }
find_optimal_route(origin_sensor, 'simulated annealing', sim_anneal_kwargs)
```

### Other functions

Two extra functions were created in the main module for purposes of running all the algorithms for all the sensors so that we have a complete solutions file. These are the `run_all()` function which executes all the algorithms for all the nodes and `get_missing_solutions()` which gets the missing solutions for each algorithm in a dictionary format.

In [70]:
def run_all():
    """
    run_all()
    executes the find_optimal_route() function for all sensors using all algorithms
    for purposes of storing the best solutions for each in the solutions file.
    returns: nothing
    """
    # algorithms to be executed
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    # all sensors except the 2 base stations
    sensors = list(x_y_data.keys())[:-2]
    # for each method, find the best path for each sensor
    for method in methods:
        for sensor in sensors:
            # ignore situations where no best paths will be found. This is typical with DGA and it will kill the function
            try:
                print(f"Sensor {sensor}")
                find_optimal_route(sensor, method)
            except:
                pass        

    return

In [71]:
#run_all()

In [72]:
def get_missing_solutions():
    """
    get_missing_solutions()
    for each algorithm, it gets the nodes that dont have any documented
    solutions to any of the based stations. This is based on the records
    found in the solutions json file.
    returns: a dictionary containing the missing solution for each algorithm
    """
    missing_solutions = {}
    sensors = list(x_y_data.keys())[:-2]
    solutions = load_solutions()

    for algorithm in solutions.keys():
        for sensor in sensors:
            if solutions[algorithm].get(f"Node-{sensor}") == None:
                if algorithm in missing_solutions.keys():
                    missing_solutions[algorithm].append(sensor)
                else:
                    missing_solutions[algorithm] = []
                    missing_solutions[algorithm].append(sensor)
    if not missing_solutions:
        missing_solutions = "All solutions available"

    return missing_solutions    

In [73]:
get_missing_solutions()

'All solutions available'

## Results

#### Discrete Genetic Algorithm

The algorithm performs well but due to its mechanism of searching for randomly generating paths, the population initialization stage, a high number of routes need to be generated to get some valid solutions by the time we are filtering out the invalid solutions. A satisfactory number of chromosomes required for standard performance (fast time and relatively good solutions) is 100,000 nodes. This is a process that could be improved by used of a graph indicating paths with connectivity. This will increase the speed of the algorithm and remove the need to initializing a high number of chromosomes. This is an improvement that we intend to include in future iterations of the algorithm for time and resource efficiency.

#### Ant Colony Optimization

#### Dijkstra's Algorithm

This application of Dijkstra's algorithm, factoring in cost, latency, and bandwidth measurements, efficiently optimizes the routing on the network. With added improvements in modularity, visualization, and documentation, it would be far more reliable and user-friendly. The algorithm performed very well due to its next node selection mechanism. For this reason, the results from the algorithm were consistent.

#### Simulated Annealing

The algorithm has a good performance for two unique implementations:<br>
1. The means of constructing the new paths without having to test invalid paths
2. The performance improvement checker which checks the significance of the improvement in the course of several iterations.

In the future, we would like to add of some extra improvement to the algorithm. This is variation of the cooling rate over time. Since the algorithm is more exploratory during the initial phases (can accept worse off solutions), we will have a slow cooling rate for proper exploration to occur but progressively, increase the cooling rate as we settle to a certain range of solutions just to save of time since at that point the algorithm rarely accepts anything that is worse off.


### Algorithm Performance Evaluation

The various algorithms that we implemented were evaluated against the following metrics:
- Execution time
- Solution quality

The performances were then visualized in a bar chart and table respectively. The code implementing performance of the algorithms is found in the [Mertics Module](./metrics.ipynb)

#### Execution Time

This metric calculates the time taken in seconds for an algorithm to execute. This measurement when the same type of problem is passed into the various algorithms, for example, getting the best path to any base station from node 43 will be passed in to all the algorithms. The time library was imported to capture execution start time and end time and this the execution time calculated as follows:

`
execution time = execution stop time – execution start time
`

This information is recorded against the algorithm name in a dictionary. After all the executions, the dictionary is used to build a bar chart which shows execution time in seconds as in the figure below.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/metrics%20chart.png?raw=true)

The function that fetches the execution time performance is shown below. To execute run `get_algorithm_time_metrics()`

In [91]:
def get_algorithm_time_metrics():
    """
    get_algorithm_time_metrics()
    this function compares the performance of the various algorithms 
    in terms of execution time and plots them on a bar graph. This is done
    by setting a timer at the beginning of execution of an algorithm and stopping
    it at the end of the execution. The duration is stored in a dictionary against
    the algorithm name as a key in the stop_execution() function. These functions
    are found in the found in the metrics module. The durations are plotted on a
    bar graph for visualization purposes.
    returns: nothing
    """
    # choose any random node to start from with the exception of the base stations
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    start_node = choice(np.array([i for i in x_y_data.keys() if i not in dest_nodes]), 1, replace=False)[0]
    # list of algorithms whose time performance will be compared
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    for method in methods:
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"executing {method}...")
        # start execution timer
        start_execution()
        # run the algorithm
        find_optimal_route(start_node, method)
        # stop the timer and record the results
        stop_execution(method)
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # plot the recorded results
    plot_algorithm_execution_time()

    return

In [92]:
#get_algorithm_time_metrics()

#### Solution Quality

The best paths from a sensor to a base station may vary for various algorithms. These paths may have varying absolute costs or similar absolute costs. This metric analyses the best paths in terms of the absolute cost and ranks them such that those that have a high absolute cost are ranked higher. But this metric being done once may not reflect the consistent generation of quality solutions. To take care of this, we set the algorithm to run several problems and for each problem rank them. At the end, ranks for each of the problems are ranked to come up with an aggregated rank of the best solution which are represented in a table as shown below.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/metrics%20table.png?raw=true)

In the figure above, Dijkstra and simulated annealing got the best solutions and tied in terms of their performance ranking. The solution quality evaluation is implemented using the function below. To execute run `get_algorithm_accuracy_metrics()` with the number of trials passed in as an argument (Default is 4).

#### Further performance comparisons

When the two ACO algorithms were run, along with all the other algorithms, it came to our attention that they perform quite differently. Hence we have decided to carry out another, less formal performance analysis.

Below is a table of timings (Table 1) on how long the runtime was on optimization of routes from all 150 nodes to one of the base stations. Where applicable, same parameters (ant number, iterations) were set. Each algorithm was run 150 times for every node as a source parameter, and an aggregate time has been indicated.

![Alt](images/Algo_Timings.JPG)


Table 1. Algorithm timings for a full run of each algorithm.

The leader, as expected, was the Dijkstra’s Algorithm, closely trailed by Simulated Annealing. It proved difficult to pinpoint concrete reasons why there was a discrepancy between the two ACO implementations. It seems that ACO (SM) must have been much more exploratory than ACO (RH).

In [98]:
def get_algorithm_accuracy_metrics(trials=4):
    """
    get_algorithm_accuracy_metrics()
    ranks the algorithms in terms of the quality of the solutions they provide.
    The ranking is done by running the several runs of all algorithms, ranking 
    the individual ranks then ranking the ranks in order to get a wholistic rank
    of the algorithms. The ranks are then displayed on a table. Rank ties 
    could exist in some scenarios
    trials (optional): number of times you want to run the algorithm to get an
    aggregated rank for more consistency in performance ranking
    returns: nothing
    """
    # choose any random node to start from with the exception of the base stations
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    # list of algorithms whose time performance will be compared
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    accuracy_performance = []
    # initialize the sum of ranks for overall ranking
    rank_sum = np.zeros(len(methods))
    trial_no = trials
    # create a store for the final rank data
    overall_performance = []
    for trial in range(trial_no):
        start_node = choice(np.array([i for i in x_y_data.keys() if i not in dest_nodes]), 1, replace=False)[0]
        print(f"trial {trial + 1}...")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        path_costs = []
        for method in methods:      
            print(f"executing {method}; trial {trial + 1}...")
            '''
            run the algorithm and get the solutions data with contains 
            the albolute cost which is used to dictate the quality of 
            the solution. The higher the cost the better the rank
            '''
            if method == "discrete genetic algorithm":
                results, solutions = discrete_genetic_algorithm(start_node)
            elif method == "dijkstras algorithm":
                results, solutions = dijkstra_algorithm(start_node)
            elif method == "ant colony optimization":
                results, solutions = ant_colony_optimization(start_node)
            elif method == "simulated annealing":
                results, solutions = simulated_annealing(start_node)
            path_costs.append(float(solutions[0]['absolute path cost']))
        # rank the path costs. negate the costs so that ranking is done in descending order
        ranked_costs = stat.rankdata([-cost for cost in path_costs])
        accuracy_performance.append(list(map(int, ranked_costs)))
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    #compile the individual trial ranks into a single wholistic rank
    for ranks in range(len(accuracy_performance)):
        for method in range(len(methods)):
            rank_sum[method] += accuracy_performance[ranks][method]
    # rank the sum of the individual ranks
    rank_data = list(map(int,stat.rankdata(rank_sum)))
    for i in range(len(rank_data)):
        overall_performance.append([rank_data[i], methods[i]])
    # tabulate the overall rank
    show_algorithm_accuracy_rank(overall_performance)

    return

In [99]:
#get_algorithm_accuracy_metrics()

#### Evaluation

After running several tests, we had the following results for our algorithms:
- Execution time metric:
    1. Dijkstra’s Algorithm (< 1 second)
    2. Simulated Annealing Algorithm (1-5 seconds)
    3. Discrete Genetic Algorithm (20 – 60seconds)
    4. Ant’s Colony Optimization Algorithm (200-250 seconds)

The indicated times ranges are where most execution times lie for round of algorithm execution. They vary based on the kind of randomly generated solutions in the cases of the stochastic algorithms like Simulated Annealing and Discrete Genetic Algorithm. These times also vary based on the **kwargs provided e.g. the population size in DGA or the number of ants in ACO. These performances were run against the default parameters which had been pre-tested to ensure optimal output.
Based on the results, Dijkstra’s Algorithm is the best algorithm for this problem in terms of time since executes in the shortest time consistently.

- Solution quality metric:
    1. Dijkstra’s Algorithm
    2. Simulated Annealing
    3. Discrete Genetic Algorithm
    4. Ant Colony Optimization

The ranks were done based on 4 trials for each algorithm, and for each trial, the best path from a particular node was tried for each algorithm. There were times when DGA emerged the best because it is dependent on the randomly generated initial paths and the permutation with is more of a trial and error. The above ranks were derived from several executions and the performance consolidated to the above.
Based on the results, Dijkstra’s Algorithm performed best for this problem because it consistently gave good results. The upside of Dijkstra’s algorithm is the logic of evaluating the next best path in the trail. This made the algorithm always get to the base station from a node in a particular route even when run several times. This consistency makes it a reliable algorithm for good solutions alongside the quick execution time as discussed above.

#### Tradeoff between time and quality

These rankings were based on several tests but again they vary based on the **kwargs provided. At the end, for stochastic algorithms, there is a tradeoff between time and quality. For example, if you initialize many ants in the ACO algorithm, more time will be used to explore the search trail and therefore to get the best path and you might get better solutions.

### Alternative Solution for ACO

The ACO as provided above finds plethora of solutions; it is, however, performing at low speed (we came to conclusion that it is more exploratory). The timings comparison has been provided at the ACO documentation section.

The ACO(2).ipynb - also referred here as ACO (RH) provides an alternative, although it is not integrated with the visualisations and formal performance analysis above. it also saves a solution file using an objective function as defined in our "Objective Function" section rather than parsing it into separate values of badnwidth and latencies.

This algorithm was used in a basic statistical analysis of solutions files by a GenAI, which calculated the optimal ACO parameters for us, providing justifications (prompts and outputs cited above).

Figure 111 peresents the flowchart of the ACO algorithms.


![Alt](images/ACO_flowchart.jpeg)


Figure 111. Flowchart of Ant Colony Optimisation algorithms.
<br>


In [105]:
#%run ACO(2).ipynb

<br>

The above algorithm creates a solutions file called solutions.txt; it is not a 100% Jason format as it uses an apostrophy instead of a double quotation mark. The main reason for the algorithm development was to provide an alternative, which would behave differently. It is different in terms of performance, but should provide similar solutions.

Below is the link to the code for the ACO algorithm.

[Alternative Ant Colony Optimization](./ACO(2).ipynb)

## Conclusion

## References

Rosenberg, M., 2024. The Distance Between Degrees of Latitude and Longitude. Available from: https://www.thoughtco.com/degree-of-latitude-and-longitude-distance-4070616 [Accessed 6th January 2024] <br>
Scripts, M. T., 2024. Calculate distance, bearing and more between Latitude/Longitude points. Available from: https://www.movable-type.co.uk/scripts/latlong.html [Accessed 6th January 2024] <br>
Story, R., 2013. Folium. Available from: https://python-visualization.github.io/folium/latest/ [Accessed 6th January 2024] <br>


## Appendix

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/appendix%20a.png?raw=true)<br>
*APPENDIX A: Map with the best path layer toggled*

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/appendix%20b.png?raw=true)<br>
*APPENDIX B: Map with possible routes only toggled*